In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
path = r'/content/drive/MyDrive/MyStudy/NCS/VNU/01-2023/MyJournal/202412-EAI CanTho/dataset/file_data_1.csv'
data = pd.read_csv(path)
data.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,37.887028,1687747.1,9.3,63.5,34.048221,34.048221,0.0,0.0,0.0,0.0,...,332.122451,817.5,5.235219e-03,5.5,24.333355,469.692075,2.092362e+05,0.9,38.5,Benign
1,48.272384,128041.4,10.4,97.8,39966.184035,39966.184035,0.0,0.0,0.0,0.0,...,694.907070,636.4,1.665200e+08,13.5,36.393289,983.968335,4.842123e+05,1.0,244.6,Benign
2,48.095663,101026.4,8.8,137.7,13.947935,13.947935,0.0,0.0,0.0,0.0,...,1302.117190,919.3,2.724581e-02,5.5,47.214170,1841.471789,2.063959e+06,0.9,38.5,Benign
3,46.342407,1301464.8,4.8,168.3,34.223130,34.223130,0.0,0.0,0.0,0.0,...,0.428562,64.8,3.804951e-02,5.5,11.472815,0.606078,9.315556e-01,0.2,38.5,Benign
4,88.807750,5290348.6,7.1,63.6,61.443831,61.443831,0.0,0.0,0.0,0.0,...,101.938534,1227.3,1.833391e-03,5.5,54.184531,144.162858,5.276998e+04,0.2,38.5,Benign


In [ ]:
# Chuẩn bị dữ liệu
X, y = data.drop('label', axis=1), data['label']
scaler = StandardScaler()
le = LabelEncoder()
X_scaled = scaler.fit_transform(X)
y_encoded = le.fit_transform(y)

In [ ]:
X.shape

(466866, 46)

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [ ]:
len(le.classes_)

8

In [ ]:
from tensorflow.keras.utils import to_categorical
# Chuyển y thành dạng one-hot để phân loại
y_train = to_categorical(y_train.squeeze())
y_test = to_categorical(y_test.squeeze())

n_input = 100
n_features = X.shape[1]
train_generator = TimeseriesGenerator(X_train, y_train, length=n_input, batch_size=512)
test_generator = TimeseriesGenerator(X_test, y_test, length=n_input, batch_size=512)
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               89600     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 2

In [ ]:
#huấn luyện
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=test_generator)
# Đánh giá mô hình trên tập kiểm tra
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
730/730 [==============================] - 269s 365ms/step - loss: 8381098496.0000 - accuracy: 0.5064 - val_loss: 197397552.0000 - val_accuracy: 0.7259
Epoch 2/10
730/730 [==============================] - 265s 363ms/step - loss: 299131648.0000 - accuracy: 0.5446 - val_loss: 40828808.0000 - val_accuracy: 0.7259
Epoch 3/10
730/730 [==============================] - 269s 368ms/step - loss: 87438384.0000 - accuracy: 0.5428 - val_loss: 5451044.5000 - val_accuracy: 0.7259
Epoch 4/10
730/730 [==============================] - 269s 369ms/step - loss: 3243975.7500 - accuracy: 0.5625 - val_loss: 489630.4062 - val_accuracy: 0.7259
Epoch 5/10
730/730 [==============================] - 263s 360ms/step - loss: 22074544.0000 - accuracy: 0.5082 - val_loss: 493127.8125 - val_accuracy: 0.3184
Epoch 6/10
730/730 [==============================] - 262s 359ms/step - loss: 11526781.0000 - accuracy: 0.5998 - val_loss: 221091.9531 - val_accuracy: 0.7259
Epoch 7/10
730/730 [========================

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
#tính RMSE và MSE
y_pred = model.predict(test_generator)
# predictions = np.argmax(predictions, axis=1)
#tính toán các metrics
y_test_trimmed = y_test[n_input:]
print("============================================")
mse = mean_squared_error(y_test_trimmed, y_pred)
mae = mean_absolute_error(y_test_trimmed, y_pred)
r2 = r2_score(y_test_trimmed, y_pred)
rmse = np.sqrt(mse)
print("Các chỉ số của dữ liệu: ")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("============================================")

2915/2915 [==============================] - 139s 47ms/step


ValueError: Input contains NaN.

In [ ]:
import time
start_time = time.time()
model.fit(train_generator, epochs=1, validation_data=test_generator)
end_time = time.time()
print(f"Thời gian huấn luyện: {end_time - start_time} giây")

  138/11669 [..............................] - ETA: 59:36 - loss: nan - accuracy: 0.0245

KeyboardInterrupt: 

In [ ]:
path = r'/content/drive/MyDrive/MyStudy/NCS/VNU/01-2023/MyJournal/202412-EAI CanTho/dataset/file_data_2.csv'
data_streaming = pd.read_csv(path)
data_streaming.head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,4.191015,154558.9,5.0,69.9,46.475504,46.475504,0.0,0.0,0.0,0.0,...,520.209290,1243.5,7.072997e-03,5.5,29.846840,735.687033,459310.280798,0.9,38.5,Benign
1,36.346295,2306740.2,6.0,97.6,63.348952,63.348952,0.0,0.0,0.0,0.0,...,593.112705,1369.2,1.512504e-03,5.5,52.567486,838.788031,511228.463201,0.7,38.5,Benign
2,29.641402,336366.2,9.3,146.6,30.376965,30.376965,0.0,0.0,0.0,0.0,...,22.440217,92.5,1.665181e+08,13.5,12.103813,31.832287,611.283761,1.0,244.6,Benign
3,16.318995,34548.1,7.1,99.1,25.329346,25.329346,0.0,0.0,0.0,0.0,...,43.187168,130.4,1.734562e-02,5.5,13.317451,61.075879,2820.248702,0.7,38.5,Benign
4,72.397705,172346.0,6.0,63.2,17.369328,17.369328,0.0,0.0,0.0,0.0,...,526.529113,1210.4,1.665231e+08,13.5,25.479511,745.257281,410803.929629,1.0,244.6,Benign


In [ ]:
# Chuẩn bị dữ liệu
X, y = data_streaming.drop('label', axis=1), data_streaming['label']
scaler = StandardScaler()
le = LabelEncoder()
X_scaled = scaler.fit_transform(X)
y_encoded = le.fit_transform(y)

In [ ]:
# Incremental Learning
import time

model_incremental = Sequential()
model_incremental.add(LSTM(128, activation='relu', input_shape=(n_input, n_features)))
model_incremental.add(Dropout(0.2))
model_incremental.add(Dense(64, activation='relu'))
model_incremental.add(Dropout(0.2))
model_incremental.add(Dense(32, activation='relu'))
model_incremental.add(Dropout(0.2))
model_incremental.add(Dense(len(le.classes_), activation='softmax'))
model_incremental.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
Wsave = model.get_weights()
model_incremental.set_weights(Wsave)


num_samples_total = X_train.shape[0]
batch_size = 256
epochs_per_batch = 5

time_all = []
num_batches = num_samples_total // batch_size
for batch_idx in range(num_batches):
    # Chia dữ liệu theo từng batch
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size
    X_batch, Y_batch = X_train[start_idx:end_idx], y_train[start_idx:end_idx]
    train_generator = TimeseriesGenerator(X_batch, Y_batch, length=n_input, batch_size=32)

    # Huấn luyện trên batch hiện tại
    print(f"Training on batch {batch_idx + 1}/{num_batches}...")
    start = time.time()
    model_incremental.fit(train_generator, epochs=epochs_per_batch, verbose=1, batch_size=batch_size)
    end = time.time()
    time_all.append(end - start)



Training on batch 1/1458...
Epoch 1/5


In [ ]:
# Đánh giá mô hình
print("\nFinal Evaluation:")
test_generator = TimeseriesGenerator(X_test, y_test, length=n_input, batch_size=32)
loss, mae = model_incremental.evaluate(test_generator, verbose=1)
print(f"Loss: {loss}, MAE: {mae}")
print('Time: ', np.sum(time_all))


Final Evaluation:
2915/2915 [==============================] - 149s 51ms/step - loss: 0.8807 - accuracy: 0.7259
Loss: 0.8806945085525513, MAE: 0.7258614301681519
Time:  11354.406195402145
